TensorFlow Lite Conversion
---
Convert the full Keras model into a smaller TensorFlow Lite model file. Then, read in the raw hex bytes from the model file and write them to a separate C header file as an array.

In [1]:
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models
from scipy import stats
import c_writer

In [3]:
# Settings
models_path = 'models'  # Where we can find the model files (relative path location)
keras_model_name = 'my_model'           # Will be given .h5 suffix
tflite_model_name = 'my_model'          # Will be given .tflite suffix
c_model_name = 'my_model'               # Will be given .h suffix1

In [4]:
# Load model
model = models.load_model(join(models_path, keras_model_name) + '.h5')

In [5]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open(join(models_path, tflite_model_name) + '.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\fede-\AppData\Local\Temp\tmpx2zn68xl\assets


1548

In [6]:
# Construct header file
hex_array = [format(val, '#04x') for val in tflite_model]
c_model = c_writer.create_array(np.array(hex_array), 'unsigned char', c_model_name)
header_str = c_writer.create_header(c_model, c_model_name)

In [7]:
# Save C header file
with open(join(models_path, c_model_name) + '.h', 'w') as file:
    file.write(header_str)

Test Inference
---
Get known good values from the model for normal and anomaly samples to compare against C++ implementation.

In [8]:
# Saved Numpy test samples file location
sample_file_path = 'test_samples'
sample_file_name = 'normal_anomaly_samples'  # Will be given .npz suffix

max_measurements = 128

In [9]:
# Load test samples
with np.load(join(sample_file_path, sample_file_name) + '.npz') as data:
    normal_sample = data['normal_sample']
    anomaly_sample = data['anomaly_sample']
print(normal_sample.shape)
print(anomaly_sample.shape)
print(normal_sample[:5])

(200, 3)
(200, 3)
[[0.054778 0.041602 0.989359]
 [0.056852 0.041724 0.990884]
 [0.054412 0.042395 0.990762]
 [0.054351 0.040138 0.991372]
 [0.056425 0.041419 0.98759 ]]


In [10]:
# Test extracting features (median absolute deviation) using SciPy
sample = normal_sample[0:max_measurements]                  # Truncate to 128 measurements
normal_x = stats.median_abs_deviation(sample)  # Calculate MAD
sample = anomaly_sample[0:max_measurements]
anomaly_x = stats.median_abs_deviation(sample)
print("Normal MAD:", normal_x)
print("Anomaly MAD:", anomaly_x)

Normal MAD: [0.0006405 0.0006405 0.000732 ]
Anomaly MAD: [0.0356445 0.029785  0.0126955]


In [11]:
# Perform inference and find MSE with normal sample
input_tensor = normal_x.reshape(1, -1)
pred = model.predict(input_tensor)
mse = np.mean(np.power(normal_x - pred, 2), axis=1)
print("Prediction:", pred)
print("MSE:", *mse)

1/1 [==============================] - 0s 91ms/step
Prediction: [[0.00069822 0.00062901 0.00075837]]
MSE: 1.3861785782522298e-09


In [12]:
# Perform inference and find MSE with anomaly sample
input_tensor = anomaly_x.reshape(1, -1)
pred = model.predict(input_tensor)
mse = np.mean(np.power(anomaly_x - pred, 2), axis=1)
print("Prediction:", pred)
print("MSE:", *mse)

1/1 [==============================] - 0s 21ms/step
Prediction: [[0.00490619 0.0062596  0.00061944]]
MSE: 0.0005480397124090671
